# Praktični primer 1 - Priporočanje filmov

V tej enoti se boste seznanili z uporabo ogrodja NVIDIA RAPIDS za reševanje realnih problemov. V tem praktičnem primeru boste najprej naložili podatkovno zbirko s podatki o ocenah filmov. Nato boste podatkovno zbirko obdelali in s pomočjo algoritmov nad grafi implementirali preprost priporočilni sistem za priporočanje filmov. Skozi cel proces implementacije boste uporabljali knjižnice ogrodja NVIDIA RAPIDS (cuDF in cuGraph).

**Preden začnemo, izvedite spodnjo celico, ki vam namesti ogrodje NVIDIA RAPIDS v okolje Google Colab.**

In [1]:
!rm -rf scripts
!rm -rf rapids-csp-utils
!mkdir scripts

install_script = """
#/bin/bash

echo "Installing NVIDIA RAPIDS to Google Colab ..."

# Check GPU
echo "Checking GPU ..."
nvidia-smi

# Clone Git repository
echo 'Cloning NVIDIA RAPIDS Git repository (https://github.com/rapidsai/rapidsai-csp-utils.git)...'
rm -rf rapidsai-csp-utils
git clone https://github.com/rapidsai/rapidsai-csp-utils.git

# Install using pip
echo 'Installing NVIDIA RAPIDS using pip ...'
python /content/rapidsai-csp-utils/colab/pip-install.py

# Post-install check
echo "Checking NVIDIA RAPIDS installation ..."
python /content/scripts/post-install-check.py
"""

post_install_script = """
import cudf
import cuml
import cugraph
import cuspatial
import cuxfilter

print(f\'cuDF: {cudf.__version__}\\ncuML: {cuml.__version__}\\ncuGraph: {cugraph.__version__}\\ncuSpatial: {cuspatial.__version__}\\ncuxfilter: {cuxfilter.__version__}\')
"""

!echo "{install_script}" >> ./scripts/install-rapids-colab.sh
!echo "{post_install_script}" >> ./scripts/post-install-check.py

!bash scripts/install-rapids-colab.sh

Installing NVIDIA RAPIDS to Google Colab ...
Checking GPU ...
Mon Jul  8 11:18:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----

## Prenos in nalaganje podatkovne zbirke

Najprej prenesite podatkovno zbirko [MovieLens25M](https://grouplens.org/datasets/movielens/25m/) in jo razširite. MovieLens25M vsebuje 25 milijon ocen in 1 milijon oznak za 62.000 filmov s strani 162.000 uporabnikov, zato lahko pričakujemo ogromen graf. Za priporočanje bosta pomembni datoteki ``ratings.csv`` in ``movies.csv``, ki ju bomo najprej naložili v cuDF DataFrame potem pa združili po skupnem ključu ``movieId``.

In [2]:
# prenos podatkovne zbirke
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

# razširjanje datoteke
!unzip ml-25m.zip

--2024-07-08 11:22:56--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  60.6MB/s    in 4.4s    

2024-07-08 11:23:00 (56.2 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [3]:
import cudf

# branje podatkov
ratings = cudf.read_csv('ml-25m/ratings.csv')
movies = cudf.read_csv('ml-25m/movies.csv')

%time df = cudf.merge(ratings, movies, on='movieId')

# izpis prvih 5 vrstic
df.head()

CPU times: user 82.2 ms, sys: 18.3 ms, total: 101 ms
Wall time: 192 ms


,userId,movieId,rating,timestamp,title,genres
0,4,2918,3.5,1573943873,Ferris Bueller's Day Off (1986),Comedy
1,4,2951,3.5,1573944499,"Fistful of Dollars, A (Per un pugno di dollari...",Action|Western
2,4,2985,4.0,1573938361,RoboCop (1987),Action|Crime|Drama|Sci-Fi|Thriller
3,4,2993,3.5,1573943471,Thunderball (1965),Action|Adventure|Thriller
4,4,3033,3.0,1573944542,Spaceballs (1987),Comedy|Sci-Fi


Podatke smo naložili v pomnilnik GPE s pomočjo cuDF, saj bomo kasneje za potrebe priporočanja zaganjali algoritma PageRank in HITS kar na GPE. To bi lahko storili tudi na CPE, vendar smo v prejšnji enoti spoznali, da to traja bistveno več časa. Tokrat implementiramo praktični primer, kjer je hitrost obdelave bistvenega pomena. Za vse pomembne operacije bomo uporabljali knjižnice, ki so del ogrodja NVIDIA RAPIDS. Govorimo torej o implementaciji celotnega delovnega toka (ang. end-to-end pipeline) s pomočjo knjižnic NVIDIA RAPIDS. Tako se izognemo tudi raznim zakasnitvam zaradi prenosa podatkov iz CPE na GPE in obratno.

## Pretvorba v graf

V podatkovni strukturi cuDF DataFrame imamo zdaj množico podatkov, ki jih moramo za potrebe priporočanja pretvoriti v graf. Za priporočanje bodo pomembni stolpci ``userId``, ``movieId`` in ``rating``. Uporabniki bodo predstavljali začetna vozlišča, filmi bodo predstavljali ciljna vozlišča, ocene pa uteži povezav med začetnimi in ciljnimi vozlišči. Uporabimo cuGraph za pretvorbo v graf, pri tem pa upoštevajmo ustrezna poimenovanja, ki jih zahteva knjižnica cuGraph (``src``, ``dst`` in ``weight``).

In [4]:
import cugraph as cg
import scipy.sparse.linalg # vključevanje te knjižnice je potrebno zaradi nekaterih operacij v cuGraph

edges = df[['userId', 'movieId', 'rating']]

# ustrezna preimenovanja stolpcev
edges = edges.rename(columns={'userId': 'src', 'movieId': 'dst', 'rating': 'weight'})

# ustvarjanje usmerjenega grafa
G = cg.Graph(directed=True)

# pretvorba v graf
%time G.from_cudf_edgelist(edges, source='src', destination='dst', edge_attr='weight', store_transposed=True)

/usr/local/lib/python3.10/dist-packages/cugraph/structure/symmetrize.py:93: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(


CPU times: user 636 ms, sys: 120 ms, total: 756 ms
Wall time: 948 ms


## Implementacija priporočanja filmov (PageRank)

Na tem mestu lahko začnemo z implementacijo priporočanja. Najprej bomo to storili z algoritmom PageRank. Pripravili bomo funkcijo za priporočanje, ki na vhodu prejme uporabnika in število priporočil. Najprej bomo pridobili za danega uporabnika pridobili filme in njegove ocene teh filmov. Nato bomo pripravili poosebitveni vektor za uporabnika, ki ga bomo uporabili v algoritmu PageRank. Rezultat algoritma PageRank bomo nato uredili po oceni algoritma padajoče in izbrali prvih K filmov, kjer je K enak vhodnemu številu priporočil.

In [20]:
def recommend_movies_pr(user_id, top_k=10):
    # filmi in ocene uporabnika
    user_ratings = df[df['userId'] == user_id]
    user_movies = user_ratings['movieId'].to_arrow().to_pylist()

    # poosebitveni vektor
    pers_vec = cudf.DataFrame({
        'vertex': user_movies,
        'values': [1.0 / len(user_movies)] * len(user_movies)
    })

    # PageRank
    pr_scores = cg.pagerank(G, personalization=pers_vec)

    # pretvorba rezultata v Pandas DataFrame; to storimo, ker cuDF še ne podpira .itertuples()
    pr_scores = pr_scores.to_pandas()

    # urejanje po oceni PageRank padajoče
    pr_scores = pr_scores.sort_values(by='pagerank', ascending=False)

    # priprava seznama priporočil, pri tem pa ne vključimo filmov, ki jih je uporabnik že ocenil
    recs = []
    for row in pr_scores.itertuples(): # zaradi tega smo zgoraj naredili pretvorbo rezultata iz cuDF v Pandas DataFrame
        if row.vertex not in user_movies: # če uporabnik filma še ni ocenil, ga dodamo na seznam priporočil
            movie_title = movies[movies['movieId'] == row.vertex]['title'].iloc[0]
            recs.append((movie_title, row.pagerank))

        if (len(recs) >= top_k): # ko imamo dovolj priporočil, končamo
            break

    return cudf.DataFrame(recs, columns=['Movie', 'Score'])

In [21]:
%time recommend_movies_pr(user_id=1337, top_k=10)

CPU times: user 112 ms, sys: 4.63 ms, total: 117 ms
Wall time: 162 ms


,Movie,Score
0,Jurassic Park (1993),0.004209
1,"Godfather, The (1972)",0.003964
2,Apollo 13 (1995),0.003677
3,"Fugitive, The (1993)",0.003511
4,Terminator 2: Judgment Day (1991),0.003429
5,Dances with Wolves (1990),0.003283
6,Raiders of the Lost Ark (Indiana Jones and the...,0.003129
7,Star Wars: Episode V - The Empire Strikes Back...,0.003126
8,True Lies (1994),0.002675
9,Aladdin (1992),0.002572


## Implementacija priporočanja filmov (HITS)

Poskusimo še implementirati priporočanje filmov z algoritmom HITS. Implementacija je zelo podobna implementaciji z algoritmom PageRank:

In [22]:
def compute_hits(G, max_iter=100):
    # HITS
    hits_scores = cg.hits(G, max_iter=max_iter)

    # normalizacija ocen HITS
    hits_scores['hubs'] /= hits_scores['hubs'].sum()
    hits_scores['authorities'] /= hits_scores['authorities'].sum()

    return hits_scores

In [23]:
def recommend_movies_hits(user_id, top_k=10):
    # filmi in ocene uporabnika
    user_ratings = df[df['userId'] == user_id]
    user_movies = user_ratings['movieId'].to_arrow().to_pylist()

    # izračun ocen HITS
    hits_scores = compute_hits(G)

    # pretvorba v Pandas DataFrame
    hits_scores = hits_scores.to_pandas()

    # izračun skupne ocene HITS
    hits_scores['combined_score'] = hits_scores['hubs'] + hits_scores['authorities']

    # urejanje po skupni oceni HITS padajoče
    hits_scores = hits_scores.sort_values(by='combined_score', ascending=False)

    # priprava seznama priporočil, pri tem pa ne vključimo filmov, ki jih je uporabnik že ocenil
    recs = []
    for row in hits_scores.itertuples(): # zaradi tega smo zgoraj naredili pretvorbo rezultata iz cuDF v Pandas DataFrame
        if row.vertex not in user_movies: # če uporabnik filma še ni ocenil, ga dodamo na seznam priporočil
            movie_title = movies[movies['movieId'] == row.vertex]['title'].iloc[0]
            recs.append((movie_title, row.combined_score))

        if (len(recs) >= top_k): # ko imamo dovolj priporočil, končamo
            break

    return cudf.DataFrame(recs, columns=['Movie', 'Score'])

In [24]:
%time recommend_movies_hits(user_id=1337, top_k=10)

CPU times: user 128 ms, sys: 16.7 ms, total: 144 ms
Wall time: 174 ms


,Movie,Score
0,Jurassic Park (1993),0.001582
1,Star Wars: Episode V - The Empire Strikes Back...,0.001539
2,Raiders of the Lost Ark (Indiana Jones and the...,0.001482
3,Terminator 2: Judgment Day (1991),0.001476
4,Back to the Future (1985),0.001447
5,"Sixth Sense, The (1999)",0.001369
6,"Godfather, The (1972)",0.001327
7,Independence Day (a.k.a. ID4) (1996),0.001316
8,Men in Black (a.k.a. MIB) (1997),0.001306
9,Fargo (1996),0.001265
